In [1]:
!pip install plot-keras-history transformers babel

     |████████████████████████████████| 1.8MB 12.5MB/s 
     |████████████████████████████████| 890kB 54.4MB/s 
     |████████████████████████████████| 2.9MB 34.7MB/s 
  Created wheel for plot-keras-history: filename=plot_keras_history-1.1.27-cp36-none-any.whl size=7333 sha256=4661fc106fcdc93bdafe2362eebfd3f7e0a046113f09306987811ae90fd70c0b
  Stored in directory: /root/.cache/pip/wheels/38/9e/e3/6cde1b6b6141b044c692d88980f26c4fe7806cc92ddd009c8c
  Created wheel for sanitize-ml-labels: filename=sanitize_ml_labels-1.0.24-cp36-none-any.whl size=7000 sha256=5c4b3125d01177db07df96c913c8d75f767445fcbc1b3daf553a256ea433344e
  Stored in directory: /root/.cache/pip/wheels/0c/3b/5d/f2fb3eff7f2fe268463b91a825816a07cd8458c4fe359f034a
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=13ea1773e8ad452defc183038a0d6e250374cd029ffed2c977b46711c6cfdef7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45


In [2]:
!git clone https://github.com/OmarSayedMostafa/Nuanced-Arabic-Dialect-Identification.git

Cloning into 'Nuanced-Arabic-Dialect-Identification'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 192 (delta 102), reused 124 (delta 46), pack-reused 0
Receiving objects: 100% (192/192), 8.40 MiB | 6.60 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [3]:
cd Nuanced-Arabic-Dialect-Identification/

/content/Nuanced-Arabic-Dialect-Identification


In [5]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import re
import random
import string
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch 
import torch.nn as nn
from sklearn.metrics import classification_report, f1_score

In [6]:
pd.set_option('display.max_colwidth', -1)

train_data_path = './cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = './cleaned_data/cleaned_test_data_for_subtask1.csv'

train_dataframe = pd.read_csv(train_data_path)
test_dataframe = pd.read_csv(test_data_path)

In [ ]:
train_dataframe.head(30)

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V0,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id
0,TRAIN_0,حاجة حلوة اكيد,Egypt,حاجة حلوة اكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,حاجه حلوه اكيد,0
1,TRAIN_1,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعدائهم,Iraq,عم بشتغلوا للشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعدائهم,عم بشتغلوا لشعب الاميركي اما نحن يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,عم بشتغلوا لشعب الاميركي يكذبوا ويغشوا ويسرقوا ويقتلو شعوبهم ويعملوا لصالح اعداهم,1
2,TRAIN_2,ابشر طال عمرك,Saudi_Arabia,ابشر طال عمرك,ابشر طال عمرك,ابشر طال عمرك,ابشر طال عمرك,2
3,TRAIN_3,منطق 2017: أنا والغريب علي إبن عمي وأنا والغريب وإبن عمي علي أخويا. #قطع_العلاقات_مع_قطر #موريتانيا_مع_قطر,Mauritania,منطق أنا والغريب علي إبن عمي وأنا والغريب وإبن عمي علي أخويا قطع العلاقات مع قطر موريتانيا مع قطر,منطق انا والغريب علي ابن عمي وانا والغريب وابن عمي علي اخويا قطع العلاقات مع قطر موريتانيا مع قطر,منطق والغريب ابن عمي وانا والغريب وابن عمي اخويا قطع العلاقات قطر موريتانيا قطر,منطق والغريب ابن عمي وانا وابن اخويا قطع العلاقات قطر موريتانيا,3
4,TRAIN_4,شهرين وتروح والباقي غير صيف ملينا,Algeria,شهرين وتروح والباقي غير صيف ملينا,شهرين وتروح والباقي غير صيف ملينا,شهرين وتروح والباقي صيف ملينا,شهرين وتروح والباقي صيف ملينا,4
5,TRAIN_5,يابنتى والله ما حد متغاظ ولا مفروس منك ولا بيحسدك انتى عره اساسا.,Syria,يابنتى والله ما حد متغاظ ولا مفروس منك ولا بيحسدك انتى عره اساسا,يابنتى واله ما حد متغاظ ولا مفروس منك ولا بيحسدك انتى عره اساسا,يابنتى واله حد متغاظ مفروس منك بيحسدك انتى عره اساسا,يابنتى واله حد متغاظ مفروس منك بيحسدك انتى عره اساسا,5
6,TRAIN_6,نفس الوقت بأكد على صاحبتي ان اي هدف هتحطه وتخططله هيبوظ والأفضل التشاؤم واننا نتوقع الأسوء دايما والفشل عشان منعشمش نفسنا ع الفاضي,Egypt,نفس الوقت بأكد على صاحبتي ان اي هدف هتحطه وتخططله هيبوظ والأفضل التشاؤم واننا نتوقع الأسوء دايما والفشل عشان منعشمش نفسنا ع الفاضي,نفس الوقت باكد على صاحبتي ان اي هدف هتحطه وتخطله هيبوظ والافضل التشام وانا نتوقع الاسوء دايما والفشل عشان منعشمش نفسنا الفاضي,باكد صاحبتي هدف هتحطه وتخطله هيبوظ والافضل التشام وانا نتوقع الاسوء دايما والفشل عشان منعشمش نفسنا الفاضي,باكد صاحبتي هدف هتحطه وتخطله هيبوظ والافضل التشام وانا نتوقع الاسوء دايما والفشل عشان منعشمش نفسنا الفاضي,0
7,TRAIN_7,م تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك URL …,Egypt,م تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك,تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك,تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك,تبطلي خرا بقا علشان مطلعهوش عليكي احترمي نفسك,0
8,TRAIN_8,ما يله دخل !,Oman,ما يله دخل,ما يله دخل,يله دخل,يله دخل,6
9,TRAIN_9,هو حلو بس يتخربط ع طلاب المدراس ليك مايغيرونه عدنا,Iraq,هو حلو بس يتخربط ع طلاب المدراس ليك مايغيرونه عدنا,هو حلو بس يتخربط طلاب المدراس ليك مايغيرونه عدنا,حلو يتخربط طلاب المدراس ليك مايغيرونه عدنا,حلو يتخربط طلاب المدراس ليك مايغيرونه عدنا,1


In [ ]:
classes_names = train_dataframe['#3_country_label'].unique().tolist()

In [ ]:
classes_names

['Egypt',
 'Iraq',
 'Saudi_Arabia',
 'Mauritania',
 'Algeria',
 'Syria',
 'Oman',
 'Tunisia',
 'Lebanon',
 'Morocco',
 'Djibouti',
 'United_Arab_Emirates',
 'Kuwait',
 'Libya',
 'Bahrain',
 'Qatar',
 'Yemen',
 'Palestine',
 'Jordan',
 'Somalia',
 'Sudan']

In [ ]:
train_x = train_dataframe['#2_tweet_clean_V0'].tolist()
train_y = train_dataframe['#classes_id'].tolist()

test_x = test_dataframe['#2_tweet_clean_V0'].tolist()
test_y = test_dataframe['#classes_id'].tolist()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bashar-talafha/multi-dialect-bert-base-arabic")
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_x,
    max_length = 80,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    test_x,
    max_length = 80,
    pad_to_max_length=True,
    truncation=True,
    return_token_type_ids=False
)

In [ ]:
# for train set
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_y)

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(test_y)

In [ ]:
# train_dataloader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#define a batch size
batch_size = 64
# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)
# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)
# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)
# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)
# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

In [ ]:
bert = AutoModel.from_pretrained("bashar-talafha/multi-dialect-bert-base-arabic")
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

In [ ]:
batch =  next(iter(train_dataloader)) 
    # # progress update after every 50 batches.
    # # push the batch to gpu
    # # batch = [r.to(device) for r in batch]
sent_id, mask, labels = batch
preds = bert(sent_id, mask)

    # print(len(preds))

    # break

In [ ]:
len(preds)

2

In [ ]:
print(preds[0].shape, preds[1].shape)

torch.Size([64, 80, 768]) torch.Size([64, 768])


In [ ]:
class BERT_Arch(nn.Module):
    def __init__(self, bert):
      super(BERT_Arch, self).__init__()
      self.bert = bert 
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      # relu activation function
      self.relu =  nn.ReLU()
      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,21)
      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):
      #pass the inputs to the model  
      _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
      x = self.fc1(cls_hs)
      x = self.relu(x)
      x = self.dropout(x)
      # output layer
      x = self.fc2(x)
      # apply softmax activation
      x = self.softmax(x)
      return x

In [ ]:
device = 'cuda'
model = BERT_Arch(bert)
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW
import numpy as np
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
y = train_dataframe['#3_country_label']
class_weights = compute_class_weight('balanced', np.unique(y), y)

print(class_weights)

[0.5533307  4.64540421 4.6671117  0.23351927 0.36638368 2.32811633
 2.32811633 1.55087252 0.77724662 4.6671117  1.16814258 0.66628546
 2.33902085 4.6671117  0.46692936 5.80675526 4.6671117  0.77603878
 1.16270303 1.5557039  2.33355585]


In [ ]:
# function for evaluating the model
def evaluate():
  print("\nEvaluating...")
  # deactivate dropout layers
  model.eval()
  total_loss, total_accuracy = 0, 0
  # empty list to save the model predictions
  total_preds = []
  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      # Calculate elapsed time in minutes.
      # elapsed = format_time(time.time() - t0)
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))
    # push the batch to gpu
    batch = [t.to(device) for t in batch]
    sent_id, mask, labels = batch
    # deactivate autograd
    with torch.no_grad():
      # model predictions
      preds = model(sent_id, mask)
      # all_predictions.append(preds)
      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
      total_preds.append(preds)
  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)
  return avg_loss, total_preds



# function to train the model
def train():
  global model
  model.train()
  total_loss, total_accuracy = 0, 0
  # empty list to save model predictions
  total_preds=[]
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))
    # push the batch to gpu
    batch = [r.to(device) for r in batch]
    sent_id, mask, labels = batch
    # clear previously calculated gradients 
    model.zero_grad()        
    # get model predictions for the current batch
    preds = model(sent_id, mask)
    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)
    # add on to the total loss
    total_loss = total_loss + loss.item()
    # backward pass to calculate the gradients
    loss.backward()
    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # update parameters
    optimizer.step()
    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()
    # append the model predictions
    total_preds.append(preds)
  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# converting list of class weights to a tensor
weights= torch.tensor(class_weights,dtype=torch.float)
# push to GPU
weights = weights.to(device)
# define the loss function
# cross_entropy  = nn.NLLLoss(weight=weights) 
cross_entropy  = nn.NLLLoss() 

# number of training epochs
epochs = 10
# set initial loss to infinite
best_valid_loss = float('inf')
# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
#for each epoch
for epoch in range(epochs):
  print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
  #train model
  train_loss, _ = train()
  #evaluate model
  valid_loss, all_prediction = evaluate()
  print(classification_report(val_y, np.argmax(all_prediction, axis=1), target_names=classes_names))
  #save the best model
  if valid_loss < best_valid_loss:
      best_valid_loss = valid_loss
      torch.save(model.state_dict(), 'saved_weights.pt')
  # append training and validation loss
  train_losses.append(train_loss)
  valid_losses.append(valid_loss)
  print(f'\nTraining Loss: {train_loss:.3f}')
  print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 20
  Batch    50  of    328.
  Batch   100  of    328.
  Batch   150  of    328.
  Batch   200  of    328.
  Batch   250  of    328.
  Batch   300  of    328.

Evaluating...
  Batch    50  of     79.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

               Egypt       0.57      0.91      0.70      1041
                Iraq       0.31      0.58      0.40       663
        Saudi_Arabia       0.30      0.59      0.40       519
          Mauritania       0.00      0.00      0.00        53
             Algeria       0.45      0.37      0.41       430
               Syria       0.21      0.12      0.15       278
                Oman       0.29      0.03      0.06       355
             Tunisia       0.20      0.02      0.03       172
             Lebanon       0.17      0.04      0.07       157
             Morocco       0.00      0.00      0.00       207
            Djibouti       0.00      0.00      0.00        27
United_Arab_Emirates       0.39      0.24      0.29       157
              Kuwait       0.00      0.00      0.00       105
               Libya       0.25      0.27      0.26       314
             Bahrain       0.00      0.00      0.00        52
       

In [ ]:
ls

 AraBert.ipynb                    LSTM_old.ipynb
 aravec/                         'prepare for trainning.ipynb'
 data/                            README.md
 environment.yml                  removed_stopwords_from_tweet.txt
'Explore Data.ipynb'              requirements.txt
'formal clitics stop words.txt'   SDG.ipynb
 LICENSE                          utilities.py
 LSTM.ipynb
